# Proyecto Final: Problema Gravitacional de los N-Cuerpos

### Desarrollado por:

* Joar Esteban Buitrago Carrillo
* Juan Sebastian Montoya Combita  

  

**Observatorio Astronómico Nacional**

2021

Las ecuaciones de movimiento de N-Partículas moviendose bajo su interacción gravitacional mutua se puede escribir en la forma.
$$m_{i}\ddot{x}_{i} = -Gm_{i} \sum_{j=1, i \neq j}^{N} \frac{m_{j}}{|x_{ij}|^{2}}$$
Donde $x_{ij}=x_{i}-x_{j}$ es el vector que apunta desde la partícula $j$ a la partícula $i$.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u
import plotly.graph_objects as go
from copy import deepcopy

##   1.   Generalización:
Implemente un código que resuelva el problema diferencial de las ecuaciones de movimiento utilizando un método Runge-Kutta de orden 4. El algoritmo debe ser lo suficientemente general para poder incluír un número arbitrario de partículas y las condiciones deben ser leídas de un archivo.






1.   Primero, se crea la clase principal para cada cuerpo



In [ ]:
class Astrobject:
  """This is an abstraction of how to implement an Astronomical Object.
  
  :param pos: The initial position of the object, defaults to np.zeros(3)*u.au.
  :type pos: A numpy array of size 3 with distance Quantity, optional.
  
  :param vel: The initial velocity of the object, defaults to np.zeros(3)*u.au/u.year.
  :type vel: A numpy array of size 3 with velocity Quantity, optional.
  
  :param mass: The initial mass of the object, defaults to 1.*u.M_sun.
  :type mass: A mass Quantity scalar, optional.
  
  """
  @u.quantity_input
  def __init__(
      self,
      pos: u.au = np.zeros(3) * u.au, 
      vel: u.au/u.year = np.zeros(3) * u.au/ u.year,
      mass: u.M_sun = 1. * u.M_sun
  ):
    """Constructor method."""
    self.pos= pos
    self.vel = vel
    assert (mass > 0)
    self.mass = mass
    self._cleardiff()
    
  def __str__(self):
    """Renders the object as a string."""
    return "Astronomical object with a mass of %s\n" % self.mass
  
  def __repr__(self):
    return self.__str__()

  def _cleardiff(self):
    """Clears and reset the diffs."""
    self.dpos = np.zeros(3) * u.au
    self.dvel = np.zeros(3) * u.au / u.year

  @u.quantity_input
  def acc(
      self,
      mass: u.M_sun, 
      distance: u.au
  ):
    """Return acceleration due to other object.
    
    :param mass: The mass of the other object.
    :type mass: A mass Quantity scalar.
    
    :param distance: The distance vector between the objects.
    :type distance:  A numpy array of size 3 with distance Quantity.
    
    :return: The acceleration vector due to other object.
    :rtype: A numpy array of size 3 with force/mass Quantity.
    
    """
    up = const.G * mass * distance
    down = (np.linalg.norm(distance)) ** 3
    return (-up/down)

  @u.quantity_input
  def add_interaction(
      self,
      other: Astrobject,
      dt: u.year
  ):
    """Add interaction using RungeKutta order 4 method.
    
    Add the interaction to the diff to before use in step function.
    
    :param other: The other object to interact with.
    :type other: class:`Astrobject`
    
    """
    orbit_mass = other.mass 
    distance = (self.pos - other.pos).to(u.au)
    
    k1 = self.vel * dt
    vk1 = self.acc(orbit_mass, distance) * dt
    
    k2 = (self.vel + (vk1/2)) * dt
    vk2 = self.acc(orbit_mass, distance + (k1/2)) * dt
    
    k3 = (self.vel + (vk2/2)) * dt
    vk3 = self.acc(orbit_mass, distance + (k2/2)) * dt
    
    k4 = (self.vel + vk3) * dt
    vk4 = self.acc(orbit_mass, distance + k3) * dt
    
    self.dpos += (k1+ 2*k2 + 2*k3 + k4) / 6
    self.dvel += (vk1 + 2*vk2 + 2*vk3 + vk4) / 6

  def step(self):
    """Add diff of velocity and position and reset those."""
    self.pos += self.dpos
    self.vel += self.dvel
    
    self._cleardiff()

    return self.pos, self.vel

2. Creación de la clase que guarda un arreglo de clases para cada cuerpo

In [ ]:
class Astrosystem:
  """This is an abstraction of an Astronomical System.
    
  :param pos_unit: The unit of the position, defaults to u.au.
  :type pos_unit: A distance unit, optional.
  
  :param vel_unit: The unit of the velocity, defaults to u.au/u.year.
  :type vel_unit: A velocity unit, optional.
  
  :param mass_unit: The unit of the mass, defaults to u.M_sun.
  :type mass_unit: A mass unit, optional.
  
  """
  @u.quantity_input
  def __init__(
      self,
      pos_unit: u.au = 1.*u.au,
      vel_unit: u.au/u.year = 1.*u.au/u.year,
      mass_unit: u.M_sun = 1.*u.M_sun
  ):
    """Constructor method."""
    self.pos_unit = pos_unit
    self.vel_unit = vel_unit
    self.mass_unit = mass_unit
    
    self.data = np.array(())
    
  def __str__(self):
    """Renders the object as a string."""
    return "Astronomical System with %d objects\n" % self.size
  
  def __repr__(self):
    """Show a pretty representation of the object."""
    return self.__str__()
    
  def loadtxt(self, *args, **kwargs):
    """Loads and parse a txt with the format pos*3, vel*3, mass.
    
    :param args: Arguments passed directly to numpy loadtxt.
    
    :param kwargs: Keyword arguments passed directly to numpy loadtxt.
    
    """
    self.data = np.loadtxt(*args, **kwargs)
    
    self.objects = np.array([
      Astrobject(
        (row[0:3] * self.pos_unit).to(u.au),
        (row[3:6] * self.vel_unit).to(u.au/u.year),
        (row[6] * self.mass_unit).to(u.M_sun)
      )
        for row in self.data
    ])
    
  @property
  def size(self):
    """Return the actual system size.
    
    :rtype: int
    
    """
    return self.data.shape[0]
    
  @staticmethod
  def distance(
      object_a: Astrobject,
      object_b: Astrobject
  ):
    """Return the distance betweem the two objects."""
    return np.linalg.norm(object_a.pos - object_b.pos)
    
  @u.quantity_input
  def advance_time(
      self,
      total_time: u.year,
      delta_time: u.year,
      interaction_distance: u.au = None,
  ):
    """Integrates the system from actual instance to total_time.
    
    The time steps are calculated using AstroPy functionality.
    
    :param total_time: The diff of time in which the system will operate.
    :type total_time: A time Quantity scalar.
    
    :param delta_time: The diff of time in which to calc the steps.
    :type total_time: A time Quantity scalar.
    
    :param interaction_distance: The minimum distance to do an interaction, 
       if None check is disabled.  Defaults to None.
    :type interaction_distance: A distane Quantity scalar or None.
    
    """
    # Calculate total time steps
    time = int ((total_time.to(delta_time.unit)) / delta_time)
    
    # Create the array of times for later analysis
    self.time_objects = np.empty((time, self.size), dtype=Astrobject)
    
    check_distance = False
    # Disable interaction distance
    if interaction_distance != None:
      check_distance = True

    for t in range(time):
      for object_a in self.objects:
        for object_b in self.objects:
          
          # Skip interacting with the same object
          if object_a is object_b:
            continue
        
          # Check if is necesary or not to check interactions
          if (not check_distance 
              or self.distance(object_a, object_b) <= interaction_distance):
            object_a.add_interaction(object_b, delta_time)
            
      # After adding all the interactions step the objects and persist them in the time array
      time_objects = np.array([], dtype=Astrobject)
      
      for body in self.objects:
        body.step()
        time_objects = np.append(time_objects, deepcopy(body))
        
      self.time_objects[t] = time_objects
      
  @u.quantity_input
  def show(
      self,
      dis_unit: u.au = 1.*u.au,
      *args,
      **kwargs
  ):
    """Creates a graph using plotly of the system."""
    if not hasattr(self, "time_objects"):
      return "Not integrated yet."
    
    fig = go.Figure()
    for i in range(self.time_objects.shape[1]):
      x, y, z = np.array([
        body.pos.to(dis_unit) 
          for body in self.time_objects[:,i]
      ]).transpose()
      fig.add_trace(
        go.Scatter3d(
          x = x,
          y = y,
          z = z,
          mode='lines'
        )
      )
    fig.show()

## 2.   Simulación sistema Sol-Tierra:
Con el fin de probar el código implementado, utilice los datos iniciales para el sistema Sol-Tierra dados en el archivo sun_earth.dat y grafique el comportamiento del sistema a lo largo de algunos años. Compruebe que la orbita no se comporta a la forma de una espiral.

1. Crear y cargar el sistema.

In [ ]:
sun_earth = Astrosystem(1.*u.m, 1.*u.m/u.s, 1.*u.kg)
sun_earth.loadtxt("sun_earth.csv", delimiter=',')

sun_earth

Astronomical System with 2 objects

2. Integrar el systema

In [ ]:
sun_earth.advance_time(5.*u.year, 1.*u.day)

3. Graficar

In [ ]:
sun_earth.show()

## 3.   Aseguramiento de Convergencia:

Con el fin de asegurar la convergencia del algoritmo, implemente una rutina dentro del programa que calcule la energía total del sistema de N-partículas. Evalue el comportamiento de la energía a lo largo de la evolución y grafique para comprobar si existe algún cambio significativo. ¿El comportamiento mejora o empeora al modificar el paso de la integración?

## 4.   Simulación Sistema estrellas alrededor de SgrA*:
Ahora utilice su código para estudiar la evolución del sistema de 13 estrellas $SO$ moviendose alrededor del agujero negro supermasivo $SgrA*$, proporcionado en el archivo SOstars.dat. Verifique también el comportamiento de la energía del sistema de 13+1 partículas. Grafique las orbitas de estas estrellas a lo largo de un periodo de 100 años.




1. Cargar los datos y crea el sistema.

In [ ]:
sgtAstar = Astrosystem(np.tan(1*u.arcsec)*8*u.kpc, np.tan(1*u.arcsec)*8*u.kpc/u.year, 1.*u.M_sun)
sgtAstar.loadtxt('sgrAstar.csv', delimiter=',')

sgtAstar

Astronomical System with 14 objects

In [ ]:
sgtAstar.advance_time(1.*u.year, 1.*u.day)

In [ ]:
sgtAstar.show(np.tan(1*u.arcsec)*8*u.kpc)